In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create network
model = Network('model')

# Block 1: nodes
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)
node3 = Queue(model, 'Queue2', SchedStrategy.PS)
node3.set_number_of_servers(2)

In [3]:
# Block 2: classes
N = [2, 0]  # Class1: 2 jobs, Class2: 0 jobs
jobclass1 = ClosedClass(model, 'Class1', N[0], node1, 0)
jobclass2 = ClosedClass(model, 'Class2', N[1], node1, 0)

In [4]:
# Set service times
node1.set_service(jobclass1, Exp.fit_mean(1.0))
node1.set_service(jobclass2, Exp.fit_mean(1.0))

node2.set_service(jobclass1, Exp.fit_mean(1.0/3.0))
node2.set_service(jobclass2, Exp.fit_mean(1.0/4.0))

node3.set_service(jobclass1, Exp.fit_mean(1.0))
node3.set_service(jobclass2, Exp.fit_mean(1.0/3.0))

In [5]:
# Block 3: routing with class switching
P = model.init_routing_matrix()

# Class1 routing: Delay -> Queue1 (stay Class1), Queue2 -> Delay (switch to Class2)
P.set(jobclass1, jobclass1, node1, node2, 1.0)  # Delay -> Queue1
P.set(jobclass1, jobclass2, node2, node3, 1.0)  # Queue1 -> Queue2 (switch to Class2)

# Class2 routing: Queue2 -> Delay (switch to Class1)
P.set(jobclass2, jobclass1, node1, node2, 1.0)  # Delay -> Queue1 (switch to Class1)
P.set(jobclass2, jobclass2, node2, node3, 1.0)  # Queue1 -> Queue2
P.set(jobclass2, jobclass1, node3, node1, 1.0)  # Queue2 -> Delay (switch to Class1)

model.link(P)

In [6]:
# Set specific state for probability calculation
n = np.array([[0,0],   # Delay: no jobs
              [1,0],   # Queue1: 1 job of Class1
              [0,1]])  # Queue2: 1 job of Class2

nodes = [node1, node2, node3]
for i in range(len(nodes)):
    nodes[i].set_state(n[i])

state = model.state()

In [7]:
# Solver options
options = Solver.defaultOptions
options.verbose = 1
options.samples = 20000
options.seed = 23000

In [8]:
# Method 1: getProbAggr - Marginal probabilities for aggregated state
results_margaggr = {}

# CTMC
try:
    solver_ctmc = CTMC(model, options)
    Pr = solver_ctmc.prob_aggr(node3)
    results_margaggr['CTMC'] = Pr
    print(f"CTMC prob_aggr: {Pr}")
except Exception as e:
    print(f"CTMC prob_aggr failed: {type(e).__name__}")

CTMC prob_aggr failed: AttributeError


In [9]:
# NC
try:
    solver_nc = NC(model, options)
    Pr = solver_nc.prob_aggr(node3)
    results_margaggr['NC'] = Pr
    print(f"NC prob_aggr: {Pr}")
except Exception as e:
    print(f"NC prob_aggr failed: {type(e).__name__}")

NC prob_aggr failed: AttributeError


In [10]:
# SSA
try:
    solver_ssa = SSA(model, options)
    Pr = solver_ssa.prob_aggr(node3)
    results_margaggr['SSA'] = Pr
    print(f"SSA prob_aggr: {Pr}")
except Exception as e:
    print(f"SSA prob_aggr failed: {type(e).__name__}")

SSA prob_aggr failed: AttributeError


In [11]:
# JMT
try:
    solver_jmt = JMT(model, options)
    Pr = solver_jmt.prob_aggr(node3)
    results_margaggr['JMT'] = Pr
    print(f"JMT prob_aggr: {Pr}")
except Exception as e:
    print(f"JMT prob_aggr failed: {type(e).__name__}")

JMT prob_aggr failed: AttributeError


In [12]:
print(f"Results: {results_margaggr}")

Results: {}


In [13]:
# Method 2: getProb - Marginal probabilities for detailed state
try:
    solver_ctmc = CTMC(model, options)
    Pr = solver_ctmc.prob(node3)
    print(f"CTMC prob: {Pr}")
except Exception as e:
    print(f"CTMC prob failed: {type(e).__name__}")

CTMC prob failed: AttributeError


In [14]:
# SSA
try:
    solver_ssa = SSA(model, options)
    Pr = solver_ssa.prob(node3)
    print(f"SSA prob: {Pr}")
except Exception as e:
    print(f"SSA prob failed: {type(e).__name__}")

SSA prob failed: AttributeError


In [15]:
# Method 3: getProbSysAggr - Joint system probabilities for aggregated state
try:
    solver_ctmc = CTMC(model, options)
    Pr = solver_ctmc.prob_sys_aggr()
    print(f"CTMC prob_sys_aggr: {Pr}")
except Exception as e:
    print(f"CTMC prob_sys_aggr failed: {type(e).__name__}")

CTMC prob_sys_aggr failed: AttributeError


In [16]:
try:
    solver_nc = NC(model, options)
    Pr = solver_nc.prob_sys_aggr()
    print(f"NC prob_sys_aggr: {Pr}")
except Exception as e:
    print(f"NC prob_sys_aggr failed: {type(e).__name__}")

NC prob_sys_aggr failed: AttributeError


In [17]:
try:
    solver_ssa = SSA(model, options)
    Pr = solver_ssa.prob_sys_aggr()
    print(f"SSA prob_sys_aggr: {Pr}")
except Exception as e:
    print(f"SSA prob_sys_aggr failed: {type(e).__name__}")

SSA prob_sys_aggr failed: AttributeError


In [18]:
try:
    solver_jmt = JMT(model, options)
    Pr = solver_jmt.prob_sys_aggr()
    print(f"JMT prob_sys_aggr: {Pr}")
except Exception as e:
    print(f"JMT prob_sys_aggr failed: {type(e).__name__}")

JMT prob_sys_aggr failed: AttributeError


In [19]:
# Method 4: getProbSys - Joint system probabilities for detailed state
try:
    solver_ctmc = CTMC(model, options)
    Pr = solver_ctmc.prob_sys()
    print(f"CTMC prob_sys: {Pr}")
except Exception as e:
    print(f"CTMC prob_sys failed: {type(e).__name__}")

CTMC prob_sys failed: AttributeError


In [20]:
# SSA
try:
    solver_ssa = SSA(model, options)
    Pr = solver_ssa.prob_sys()
    print(f"SSA prob_sys: {Pr}")
except Exception as e:
    print(f"SSA prob_sys failed: {type(e).__name__}")

SSA prob_sys failed: AttributeError
